<a href="https://colab.research.google.com/github/fafnirkyu/data-challenge-alura/blob/Semana-1/semana_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ---------------------------------------------------Semana 1-------------------------------------------------------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Limpeza dos dados

In [ ]:
!pip install pyspark==3.3.1

In [ ]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, StringType, DoubleType, StructField

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [ ]:
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip' && unzip semana-1.zip 

In [ ]:
dados = spark.read.json('/content/dataset_bruto.json')
dados.printSchema()


In [ ]:
dados.limit(5).toPandas()

In [ ]:
dadosanuncio = dados.select("anuncio.*").select("id","andar","quartos", "suites","area_total","area_util","banheiros","caracteristicas","endereco.*", "tipo_anuncio"
,"tipo_unidade","tipo_uso", "vaga", "valores.condominio", "valores.iptu", "valores.tipo", "valores.valor")

In [ ]:
dadosanuncio = dadosanuncio.where(dadosanuncio.tipo_uso=='Residencial').where(dadosanuncio.tipo_unidade=='Apartamento').where(dadosanuncio.tipo_anuncio=='Usado')

antes do filtro 89083
depois do filtro 66562

In [ ]:
dadosanuncio.limit(5).toPandas()

In [ ]:
from pyspark.sql import functions as f

In [ ]:
from pyspark.sql.types import DoubleType, StringType,IntegerType, LongType, ArrayType
dadosanuncio.printSchema()



In [ ]:

dados = dadosanuncio.withColumn('quartos', dadosanuncio['quartos'][0])\
.withColumn('suites', dadosanuncio['suites'][0])\
.withColumn('area_total', dadosanuncio['area_total'][0])\
.withColumn('area_util', dadosanuncio['area_util'][0])\
.withColumn('banheiros', dadosanuncio['banheiros'][0])\
.withColumn('vaga', dadosanuncio['vaga'][0])\
.withColumn('condominio', dadosanuncio['condominio'][0])\
.withColumn('iptu', dadosanuncio['iptu'][0])\
.withColumn('tipo', dadosanuncio['tipo'][0])\
.withColumn('valor', dadosanuncio['valor'][0])

In [ ]:
dados.toPandas()

In [ ]:
dados.printSchema()

In [ ]:
#Nesse contexto, transforme os dados das colunas 
#"quartos", "suites", "banheiros", "vaga", "area_total" e "area_util" de listas para inteiros.
dados = dados.withColumn('quartos', dados['quartos'].cast(IntegerType()))\
    .withColumn('suites', dados['suites'].cast(IntegerType()))\
    .withColumn('banheiros', dados['banheiros'].cast(IntegerType()))\
    .withColumn('vaga', dados['vaga'].cast(IntegerType()))\
    .withColumn('area_total', dados['area_total'].cast(IntegerType()))\
    .withColumn('area_util', dados['area_util'].cast(IntegerType()))

In [ ]:
dados.printSchema()

In [ ]:
#transformar apenas as informações sobre bairro e zona em colunas de nosso DataFrame.
dados = dados.drop("cidade", "estado", "latitude", "longitude", "pais", "rua", "cep")
dados.toPandas()

In [ ]:
dados = dados.filter('tipo == "Venda"')
dados.toPandas()


In [ ]:
dados.write.parquet('/content/drive/MyDrive/dadossemana1', mode='overwrite')